# Section 7 – Game Loop & Updates (dt best practices)

In Arcade, your “game loop” lives in two core callbacks:

- `on_update(self, dt)`: **update state** (movement, timers, physics).
- `on_draw(self)`: **render** the current state (no heavy logic here).

Key ideas:
- Use **pixels per second** for speeds, and multiply by **`dt`** to get frame-rate independence.
- Keep input -> **state** (velocity, targets) -> **update with dt** -> **draw**.
- For timed events/animation, use a **time accumulator**: `acc += dt`; when `acc >= period`, trigger and subtract.

We’ll demo:
1. **Time-based motion + simple animation** using a texture flip timer.
2. **Bouncing + gravity** using velocity and acceleration with `dt`.

---

# Exercise 7.1 – Timers & dt
- Change the flip rate to 12 FPS by adjusting `frame_period`.
- Make the coin move **left** and wrap on the **left** edge.
- Pause/resume animation with a key (e.g., `P`) by skipping the animation code when paused.


In [ ]:
import arcade
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
TITLE = "Motion and animation"

class DtBasics(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.ASH_GREY)

        self.robot_list = arcade.SpriteList()
        self.robot = arcade.Sprite(":resources:/images/animated_characters/robot/robot_walk0.png")
        self.robot.center_x, self.robot.center_y = 80, SCREEN_HEIGHT // 2
        self.robot_list.append(self.robot)

        self.speed_x = 180.0

        self.frames = [
            arcade.load_texture(":resources:/images/animated_characters/robot/robot_walk0.png"),
            arcade.load_texture(":resources:/images/animated_characters/robot/robot_walk1.png"),
            arcade.load_texture(":resources:/images/animated_characters/robot/robot_walk2.png"),
            arcade.load_texture(":resources:/images/animated_characters/robot/robot_walk3.png"),
        ]
        self.frame_index = 0
        self.frame_period = 0.125
        self.frame_acc = 0.0

        # wobble settings
        self.t = 0.0
        self.wobble_amplitude_deg = 4.0 # ± degrees
        self.wobble_speed_hz = 1.5 # cycles per second

    def on_draw(self):
        self.clear()